In [20]:
import matplotlib.pyplot as plt
from kneed import KneeLocator
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import folium
from collections import Counter

df = pd.read_csv("../Dati/request_cr.csv", delimiter=",", encoding = "ISO-8859-1")
to_ = list(df['biblioteca_destinazione'])
from_ = list(df['biblioteca_partenza'])
data = np.column_stack((from_, to_)).tolist()
occ = [[x, data.count(x)] for x in data]

In [37]:
df

,loan_id,data_richiesta,biblioteca_destinazione,biblioteca_partenza
0,22690933,2016-01-01 09:35:10,BONEMERSE,RIVOLTA D'ADDA
1,22690934,2016-01-01 11:40:36,ROMANENGO,RIVOLTA D'ADDA
2,22690935,2016-01-01 17:54:48,Agnadello,RIVOLTA D'ADDA
3,22690936,2016-01-01 21:10:08,SPINO D'ADDA,RIVOLTA D'ADDA
4,22690937,2016-01-01 21:11:29,SPINO D'ADDA,RIVOLTA D'ADDA
5,22690977,2016-01-02 00:24:44,BONEMERSE,BRESCIA
6,22691160,2016-01-02 09:56:13,GADESCO PIEVE DELMONA,GADESCO PIEVE DELMONA
7,22692122,2016-01-01 18:04:24,SPINO D'ADDA,PALAZZO PIGNANO
8,22692123,2016-01-01 23:54:11,RIVOLTA D'ADDA,PALAZZO PIGNANO
9,22692124,2016-01-02 17:24:25,PIADENA DRIZZONA,PALAZZO PIGNANO


In [36]:
def unique(list_): 
  
    # intilize a null list 
    unique_list = [] 
      
    # traverse for all elements 
    for x in list_: 
        # check if exists in unique_list or not 
        if x not in unique_list: 
            unique_list.append(x) 
    return unique_list
unique_occ = unique(occ)

In [44]:
origins = []
destinations = []
values = []
for i in unique_occ:
    origins.append(i[0][0])
    destinations.append(i[0][1])
    values.append(i[1])
d = {'origin' : origins, 'destination': destinations, 'num_orders': values}
dataframe = pd.DataFrame(d)
dataframe = dataframe.sort_values('num_orders', ascending = False)


,origin,destination,num_orders
135,BRESCIA,CREMA,20176
75,BRESCIA,OFFANENGO,3958
152,CREMA,BRESCIA,2540
67,BRESCIA,SONCINO,2533
226,RIVOLTA D'ADDA,CREMA,1965
66,BRESCIA,SORESINA,1936
38,BRESCIA,RIVOLTA D'ADDA,1723
207,RIVOLTA D'ADDA,BRESCIA,1641
137,CASTELLEONE,CREMA,1582
93,BRESCIA,MADIGNANO,1566


In [46]:
dataframe.to_csv("../Dati/num_orders.csv", index = False, sep = ';')